In [1]:
!conda install -c conda-forge --yes librosa

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/studio-lab-user/.conda/envs/default

  added / updated specs:
    - librosa


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.5.0                  |       h27087fc_0         2.7 MB  conda-forge
    appdirs-1.4.4              |     pyh9f0ad1d_0          13 KB  conda-forge
    audioread-3.0.0            |   py39hf3d152e_0          34 KB  conda-forge
    brotli-1.0.9               |       h166bdaf_7          18 KB  conda-forge
    brotli-bin-1.0.9           |       h166bdaf_7          19 KB  conda-forge
    brotlipy-0.7.0             |py39hb9d737c_1004         342 KB  conda-forge
    ca-certificates-2022.9.24  |       ha878542_0         150 

In [4]:
!pip install -r requirements.txt

In [28]:
!pip install sentencepiece

     |████████████████████████████████| 1.3 MB 4.5 MB/s eta 0:00:01


In [8]:
# Imported libraries
import librosa
import transformers
import torchaudio
import sentencepiece
import numpy as np
import math
import regex as re
import csv

from scipy.io.wavfile import write
from pyctcdecode import build_ctcdecoder
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
def add_background_noise(s_path, n_path, offset=0.0, SNR=10):
  sample, sr = librosa.load(s_path)  # Load audio file
  duration_s = librosa.get_duration(y=sample, sr=sr)  # Get audio file length
  noise, sr = librosa.load(n_path, offset=offset, duration=duration_s) # Load same length of noise file
  RMS_s = math.sqrt(np.mean(sample**2))  # Calculate RMS for audio sample
  RMS_n = math.sqrt(RMS_s**2/(pow(10,5/10)))  # Calculate required RMS for noise
  RMS_n_current = math.sqrt(np.mean(noise**2))  # Calculate current RMS for noise
  noise = noise*(RMS_n/RMS_n_current)  # Adjust noise
  sample_noise = sample + noise  # Add sample to noise
  sample_noise = librosa.resample(sample_noise, orig_sr=sr,target_sr=16000)
  return sample_noise

In [10]:
def asr_model(wav_file, model, processor):
    waveform, sample_rate = torchaudio.load(wav_file)
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000, resampling_method='sinc_interpolation')
    # Downsampling is required to match the model's frequency, i.e.
    # the frequency of the dataset the model was trained on.
    speech_array = resampler(waveform).squeeze().numpy()
    inputs = processor(speech_array, sampling_rate=16000, return_tensors="pt")['input_values']
    # Detach the tensors first (otherwise, they require gradients
    # to be computed); also, since we're simulating the inference
    # squeeze the resulting array at the first dimension (batch_size),
    # otherwise, leave it as is, and call batch_decode() instead.
    outputs = model(inputs)['logits'].detach().numpy().squeeze(0)
    return(processor.decode(outputs).text)

In [11]:
def translate_uk_en(model_name, ukrainian_text):
    tokenizer = { model_name : AutoTokenizer.from_pretrained(model_name) }
    model = { model_name : AutoModelForSeq2SeqLM.from_pretrained(model_name) }
    encoded = { model_name : model[model_name].generate(**tokenizer[model_name](ukrainian_text, return_tensors='pt', padding=True)) }
    translation = { model_name : [ tokenizer[model_name].decode([t for t in tensor], skip_special_tokens=True) for tensor in encoded[model_name] ] }
    return translation[model_name][0]


In [12]:
s_list = [
    ['spk_id_9235_16144789581693.wav', 'сорок дев\'ять дітей найбільша кількість дітей від початку епідемії декілька дітей в'],
    ['spk_id_11378_16144786658795.wav', 'активна розбудова міста це величезна проблема києва висотки ростуть нові дитячі садки школи та'],
    ['spk_id_10865_16144786894916.wav', 'мали право на державну допомогу в період карантину як ресторани кафе інакше німеччина ризикує втратити'],
    ['spk_id_10433_16144786840438.wav', 'уряд про негайну допомогу каже що без фінансової підтримки галузі може опинитися на межі банкрутства'],
    ['spk_id_10485_16144791102943.wav', 'серія розшифровці'],
    ['spk_id_11489_16144791126466.wav', 'свій про що'],
    ['spk_id_9206_16144786851920.wav', 'найкраще пиво і тепер коли ми змушені його просто виливати у нас душа болить пиво зварене'],
    ['spk_id_9748_16144788856849.wav', 'паспорт тут номер посвідчення особи дата закінчення терміну дії серпень двадцять першого року так'],
    ['spk_id_11049_16144791123356.wav', 'бути з в давай'],
    ['spk_id_10751_16144789434726.wav', 'сьогодні до'],
    ['spk_id_10777_16144790024495.wav', 'цього мобільні групи які є в кожній з областей беруть вакцину на обласному складі і виїжджають у закладах охорони здоров\'я який надає допомогу хворим з коронавірусної хвороб оскільки відповідно до дорожньої карти з проведення'],
    ['spk_id_10227_16144788259344.wav', 'за рахунок бізнесу звісно не подобається підприємцям але маючи необхідність тільки зараз влити в економіку два трильйони і це не останній пакет країна почала відмовлятися від економічного курсу трампа який податки тільки знижував'],
    ['spk_id_9989_16144787694828.wav', 'всі города'],
    ['spk_id_10215_16144791155880.wav', 'посадці зробити'],
    ['spk_id_9976_16144787668804.wav', 'сина для комп\'ютера'],
    ['spk_id_11457_16144790001415.wav', 'не змогли повірте ми зробили все можливе щоб україна отримала найкращу ціну за цю вакцину ми знаходимося під договором про'],
    ['spk_id_9507_16144787704673.wav', 'в схеми страждають'],
    ['spk_id_10776_16144788871212.wav', 'те саме різницею'],
    ['spk_id_11374_16144786380608.wav', 'ідею таких соціальних харчевень позичили в сусідній франції там їх сотні працюють волонтери фінансують'],
    ['spk_id_10827_16144786663913.wav', 'тіла в києва'],
    ['spk_id_11747_16144790047833.wav', 'до шістдесяти років зможуть зробити щеплення вже на п\'ятому етапі записатися на вакцинацію можна буде через інтернет та'],
]
n_list = [
    ['noise/bus_ch01.wav', 0.0],
    ['noise/cafeteria_ch01.wav', 22.0], 
    ['noise/car_ch01.wav', 13.0],
    ['noise/field_ch01.wav', 79.0],
    ['noise/hallway_ch01.wav', 222.0], 
    ['noise/kitchen_ch01.wav', 145.0],
    ['noise/living_ch01.wav', 32.0],
    ['noise/meeting_ch01.wav', 3.0],
    ['noise/metro_ch01.wav', 0.0],
    ['noise/office_ch01.wav', 162.0],
    ['noise/park_ch01.wav', 42.0],
    ['noise/resto_ch01.wav', 0.0],
    ['noise/river_ch01.wav', 85.0], 
    ['noise/square_ch01.wav', 0.0],
    ['noise/station_ch01.wav', 59.0],
    ['noise/traffic_ch01.wav', 0.0],
    ['noise/washing_ch01.wav', 150.0]
]

sr_model = Wav2Vec2ForCTC.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")
processor = Wav2Vec2ProcessorWithLM.from_pretrained("Yehor/wav2vec2-xls-r-300m-uk-with-small-lm")

nmt_model = "Helsinki-NLP/opus-mt-tc-big-zle-en"

header = ['path', 'original_transcript', 'noise_type', 'asr_transcript', 'translation']
data_translated = []



/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'snapshot_download': allow_regex. Will not be supported from version '0.12'.

Please use `allow_patterns` and `ignore_patterns` instead.
  warnings.warn(message, FutureWarning)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading the LM will be faster if you build a binary file.
Reading /home/studio-lab-user/.cache/pyctcdecode/models--Yehor--wav2vec2-xls-r-300m-uk-with-small-lm/snapshots/bbd936400e7566ba44560440aa4abd05b5983c17/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [13]:
for s_path in s_list:
  for n_path in n_list:
    noise_type = re.findall('/(\S+?)_ch01\.wav', n_path[0])[0]
    file_path = 'samples_with_noise_snr5/' + re.findall('(\S+?)\.wav', s_path[0])[0] + '_' + noise_type + ".wav"
    sr = librosa.get_samplerate(n_path[0])
    write(file_path, rate=sr, data=add_background_noise('samples/'+s_path[0], n_path[0], offset=n_path[1], SNR=5))
    asr_transcript = asr_model(file_path, sr_model, processor) 
    data_translated.append([file_path, s_path[1], noise_type, asr_transcript, translate_uk_en(nmt_model, asr_transcript)])

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
with open('translated_with_noise_snr5.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data_translated)